In [5]:
pip install python-dotenv pandas


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import requests
import time
from dotenv import load_dotenv
import os
import json
import yaml

load_dotenv() 

True

In [2]:
# Configuration
GIT_REPO_FILTERED = "files/git_repo_filtered_js_commit_date.csv"
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GRAPHQL_URL = "https://api.github.com/graphql"
HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

repos_from_csv = []

repos_with_CI = set()
invalid_repos = set()
repos_with_no_workflows = set()
repos_with_network_error = set()
repo_name_to_branch = {}

In [28]:

def read_filtered_repo_csvlist():
    df = pd.read_csv(GIT_REPO_FILTERED)

    # Split 'name' into owner and repo
    for _, row in df.iterrows():
        if '/' not in row['name']:
            print(f"Skipping invalid repo name: {row['name']}")
            continue
        owner, repo_name = row['name'].split('/', 1)
        repos_from_csv.append({
            "owner": owner.strip(),
            "name": repo_name.strip(),
            "default_branch": row['default_branch'].strip()
        })
        repo_name_to_branch[f"{owner.strip()}/{repo_name.strip()}".lower()] = row['default_branch'].strip()

    print(repos_from_csv[:5])

read_filtered_repo_csvlist()

[{'owner': 'bigbluebutton', 'name': 'bigbluebutton', 'default_branch': 'v3.0.x-release'}, {'owner': 'zabinx', 'name': 'duskrpg', 'default_branch': 'master'}, {'owner': 'apache', 'name': 'cordova-android', 'default_branch': 'master'}, {'owner': 'aws-samples', 'name': 'aws-dynamodb-examples', 'default_branch': 'master'}, {'owner': 'dgarijo', 'name': 'widoco', 'default_branch': 'master'}]


In [29]:
# Step 2: Build GraphQL query for multiple repos
def build_query_to_check_workflows(repos, start, end):
    query_parts = []
    for i, repo in enumerate(repos[start:end]):
        query_parts.append(f"""
        repo{i}: repository(owner: "{repo['owner']}", name: "{repo['name']}") {{
            workflows: object(expression: "{repo['default_branch']}:.github/workflows") {{
                ... on Tree {{
                    entries {{
                        name
                        type
                    }}
                }}
            }}
        }}
        """)
    full_query = "query { " + " ".join(query_parts) + " }"
    # print("query:", full_query)
    return full_query

In [18]:
# Step 3: Execute query and parse results
def check_workflows(repos, start, end):
    try:
        query = build_query_to_check_workflows(repos, start, end)
        response = requests.post(GRAPHQL_URL, json={"query": query}, headers=HEADERS)
        response.raise_for_status()
        data = response.json()

        # If GitHub responds with an error (403, 502, etc.)
        if response.status_code != 200:
            print(f"Skipping batch {start}:{end} (HTTP {response.status_code})")
            for repo in repos[start:end]:
                repos_with_network_error.add(f"{repo['owner']}/{repo['name']}")
            return

        data = response.json()

        for i, repo in enumerate(repos[start:end]):
            key = f"repo{i}"
            repo_data = data.get("data", {}).get(key, {})

            if not repo_data:
                invalid_repos.add(f"{repo['owner']}/{repo['name']}")
                continue
            workflows = repo_data.get("workflows")

            if workflows and workflows.get("entries"):
                repos_with_CI.add(f"{repo['owner']}/{repo['name']}")
            else:
                repos_with_no_workflows.add(f"{repo['owner']}/{repo['name']}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed for batch {start}:{end}: {e}")
        for repo in repos[start:end]:
            repos_with_network_error.add(f"{repo['owner']}/{repo['name']}")

In [20]:
start = 0
batch_size = 100
limit = len(repos_from_csv)

repos_with_CI = set()
invalid_repos = set()
repos_with_no_workflows = set()
repos_with_network_error = set()

for start in range(start, limit, batch_size):
    end = min(start + batch_size, limit)
    check_workflows(repos_from_csv, start, end)
    print(f"Processed {start} to {end}. repos_with_CI:{len(repos_with_CI)} repos_with_network_error:{len(repos_with_network_error)} repos_with_no_workflows:{len(repos_with_no_workflows)} invalid_repos:{len(invalid_repos)}")
    time.sleep(5)

Processed 0 to 100. repos_with_CI:54 repos_with_network_error:0 repos_with_no_workflows:45 invalid_repos:1
Processed 100 to 200. repos_with_CI:120 repos_with_network_error:0 repos_with_no_workflows:77 invalid_repos:3
Processed 200 to 300. repos_with_CI:152 repos_with_network_error:0 repos_with_no_workflows:144 invalid_repos:4
Processed 300 to 400. repos_with_CI:177 repos_with_network_error:0 repos_with_no_workflows:219 invalid_repos:4
Processed 400 to 500. repos_with_CI:194 repos_with_network_error:0 repos_with_no_workflows:301 invalid_repos:5
Processed 500 to 600. repos_with_CI:228 repos_with_network_error:0 repos_with_no_workflows:366 invalid_repos:6
Processed 600 to 700. repos_with_CI:259 repos_with_network_error:0 repos_with_no_workflows:435 invalid_repos:6
Processed 700 to 800. repos_with_CI:283 repos_with_network_error:0 repos_with_no_workflows:511 invalid_repos:6
Processed 800 to 900. repos_with_CI:310 repos_with_network_error:0 repos_with_no_workflows:583 invalid_repos:7
Proces

In [21]:
# Save each set to a separate file
def save_repo_names_to_file():
    with open("files/repos_with_CI.txt", "w") as f:
        for repo in (repos_with_CI):
            f.write(repo + "\n")

    with open("files/repos_with_no_workflows.txt", "w") as f:
        for repo in (repos_with_no_workflows):
            f.write(repo + "\n")

    with open("files/invalid_repos.txt", "w") as f:
        for repo in (invalid_repos):
            f.write(repo + "\n")

    with open("files/repos_with_network_error.txt", "w") as f:
        for repo in (repos_with_network_error):
            f.write(repo + "\n")

In [22]:
save_repo_names_to_file()

In [47]:
# retrying repos with network error
unchecked_repos = []
for error_repo in repos_with_network_error:
    owner, repo_name = error_repo.split('/', 1)
    unchecked_repos.append({
        "owner": owner.strip(),
        "name": repo_name.strip(),
        "default_branch": repo_name_to_branch[f"{owner.strip()}/{repo_name.strip()}"].strip()
    })

batch_size = 100
repos_with_network_error = set()

for start in range(0, len(unchecked_repos), batch_size):
    end = min(start + batch_size, limit)
    check_workflows(unchecked_repos, start, end)

print(f'repos_with_CI:{len(repos_with_CI)}, repos_with_network_error: {len(repos_with_network_error)}')


repos_with_CI:12215, repos_with_network_error: 0


In [51]:
save_repo_names_to_file()
print(f"repos_with_CI:{len(repos_with_CI)} repos_with_network_error:{len(repos_with_network_error)} repos_with_no_workflows:{len(repos_with_no_workflows)} invalid_repos:{len(invalid_repos)}")

repos_with_CI:12215 repos_with_network_error:0 repos_with_no_workflows:11965 invalid_repos:432


In [78]:
# read from saved files
repos_list_with_CI = []

with open("files/repos_with_CI.txt", "r") as file:
    for line in file:
        repos_list_with_CI.append(line.lower())

print(f"repos_with_CI:{len(repos_list_with_CI)}")

repos_with_CI:12215


In [79]:
def build_query_for_download_workflows(start, end, repos_list):
    query_parts = []
    for idx, repo_full in enumerate(repos_list[start:end]):
        owner, name = [x.strip() for x in repo_full.split("/")]
        query_parts.append(f"""
        repo{idx}: repository(owner: "{owner}", name: "{name}") {{
            object(expression: "HEAD:.github/workflows") {{
                ... on Tree {{
                    entries {{
                        name
                        type
                        object {{
                            ... on Blob {{
                                text
                            }}
                        }}
                    }}
                }}
            }}
        }}
        """)
    return "query {\n" + "\n".join(query_parts) + "\n}"

In [ ]:
def download_workflows(repos_list):
    start = 0
    batch_size = 50
    limit = len(repos_list)
    sleep_time = 3
    workflow_files = "dummy"

    for start in range(start, limit, batch_size):
        time.sleep(sleep_time)  # To respect rate limits
        print(f"Processing chunk {start} to {min(start + batch_size, limit)}")
        try:
            end = min(start + batch_size, limit)
            query = build_query_for_download_workflows(start, end, repos_list)
            response = requests.post(
                "https://api.github.com/graphql",
                json={"query": query},
                headers=HEADERS,
                timeout=30  # Optional: avoid hanging
            )
            response.raise_for_status()
            data = response.json().get("data", {})

            for idx, repo_full in enumerate(repos_list[start:end]):
                owner, repo_name = repo_full.split("/")
                repo_key = f"repo{idx}"

                if(data.get(repo_key, {}) is None or data.get(repo_key, {}).get("object") is None):
                    print(f"Data missing {repo_full}")
                    continue

                entries = data.get(repo_key, {}).get("object", {}).get("entries", [])

                if not entries:
                    print(f"No workflows found in {repo_full}")
                    continue

                save_folder = os.path.join(workflow_files, owner.strip(), repo_name.strip())
                os.makedirs(save_folder, exist_ok=True)

                for entry in entries:
                    if entry["type"] == "blob":
                        filename = entry["name"]
                        content = entry["object"]["text"]
                        file_path = os.path.join(save_folder, filename)
                        if content is not None:
                            with open(file_path, "w") as f:
                                f.write(content)

        except Exception as e:
            print(f"Error processing chunk {start}-{end}: {e}")

In [54]:
download_workflows(repos_list_with_CI)

Processing chunk 0 to 50
Processing chunk 50 to 100
Processing chunk 100 to 150
Processing chunk 150 to 200
Processing chunk 200 to 250
Processing chunk 250 to 300
Processing chunk 300 to 350
Processing chunk 350 to 400
Processing chunk 400 to 450
Processing chunk 450 to 500
Processing chunk 500 to 550
Processing chunk 550 to 600
Processing chunk 600 to 650
Processing chunk 650 to 700
Processing chunk 700 to 750
Processing chunk 750 to 800
Processing chunk 800 to 850
Processing chunk 850 to 900
Processing chunk 900 to 950
Processing chunk 950 to 1000
Processing chunk 1000 to 1050
Processing chunk 1050 to 1100
Processing chunk 1100 to 1150
Processing chunk 1150 to 1200
Processing chunk 1200 to 1250
Data missing monsternone/tmall-miao

Processing chunk 1250 to 1300
Processing chunk 1300 to 1350
Processing chunk 1350 to 1400
Processing chunk 1400 to 1450
Processing chunk 1450 to 1500
Processing chunk 1500 to 1550
Processing chunk 1550 to 1600
Processing chunk 1600 to 1650
Processing chunk

In [267]:
def total_downloaded_workflows(root_folder):
    result = []
    for parent in os.listdir(root_folder):
        parent_path = os.path.join(root_folder, parent)
        if os.path.isdir(parent_path):
            for child in os.listdir(parent_path):
                child_path = os.path.join(parent_path, child)
                if os.path.isdir(child_path):
                    result.append(f"{parent}/{child}".lower().strip())
    return result

# Example usage:
root = "workflow_files"
total_downloaded_workflow_list = total_downloaded_workflows(root)

print("\nTotal:", len(total_downloaded_workflow_list))


Total: 12210


In [ ]:
def get_unretrieved_repos():
    unretrieved_repos_list = []
    for r in repos_list_with_CI:
        if r not in total_downloaded_workflow_list:
            unretrieved_repos_list.append(r)
    
    return unretrieved_repos_list


In [70]:
unretrieved_repos_list = get_unretrieved_repos()

In [72]:
# download unretrieved repos: for network issues or other issues
download_workflows(unretrieved_repos_list)

Processing chunk 0 to 5
Data missing monsternone/tmall-miao

Data missing hjyssg/shigureader

Data missing jayofelony/pwnagotchi

Data missing nulldev/spendenr-ai-d

Data missing cpinitiative/ide



In [263]:
ci_datas = []
repos_with_missing_ci_datas = []

def get_ci_data():
    index = 0
    for repo in total_downloaded_workflow_list:
        
        index += 1
        if(index%20 == 0):
            print(f"Processing {index} out of {len(total_downloaded_workflow_list)}")
        
        try:
            time.sleep(1)

            OWNER, REPO = repo.split('/', 1)
            BRANCH = repo_name_to_branch[f"{OWNER}/{REPO}".lower()]

            # Get workflow runs directly (this gets the latest commit info too)
            runs_url = f"https://api.github.com/repos/{OWNER}/{REPO}/actions/runs"
            params = {"per_page": 1} # Ignoring main/master branch, as latest commit could be in any branch

            runs_resp = requests.get(runs_url, headers=HEADERS, params=params)
            runs_resp.raise_for_status()
            runs_data = runs_resp.json()

            if "workflow_runs" in runs_data and runs_data["workflow_runs"]:
                run = runs_data["workflow_runs"][0]
                latest_sha = run.get("head_sha")  # Get SHA from workflow run

                ci_datas.append({
                    "repo": f"{OWNER}/{REPO}",
                    "branch": run.get("head_branch"),
                    "default_branch": BRANCH,
                    "commit": latest_sha,
                    "workflow_name": run.get("name"),
                    "run_id": run.get("id"),
                    "status": run.get("status"),
                    "conclusion": run.get("conclusion"),
                    "event": run.get("event"),
                    "url": run.get("html_url"),
                    "start_time": run.get("run_started_at"),
                    "end_time": run.get("updated_at"),
                    "path": run.get("path"),
                })

            else:
                repos_with_missing_ci_datas.append(f"{OWNER}/{REPO}")
                
        except Exception as e:
            print(f"Error processing {OWNER}/{REPO}: {e}")
            repos_with_missing_ci_datas.append(f"{OWNER}/{REPO}")

In [268]:
get_ci_data()

df_ci_data = pd.DataFrame(ci_datas)


df_ci_data.to_csv("files/ci_data.csv", index=False)

with open("files/repos_with_missing_ci_datas.txt", "w") as f:
    for repo in (repos_with_missing_ci_datas):
        f.write(repo + "\n")

Processing 20 out of 12210
Processing 40 out of 12210
Processing 60 out of 12210
Processing 80 out of 12210
Processing 100 out of 12210
Processing 120 out of 12210
Processing 140 out of 12210
Processing 160 out of 12210
Processing 180 out of 12210
Processing 200 out of 12210
Processing 220 out of 12210
Processing 240 out of 12210
Processing 260 out of 12210
Processing 280 out of 12210
Processing 300 out of 12210
Processing 320 out of 12210
Processing 340 out of 12210
Processing 360 out of 12210
Processing 380 out of 12210
Processing 400 out of 12210
Processing 420 out of 12210
Processing 440 out of 12210
Processing 460 out of 12210
Processing 480 out of 12210
Processing 500 out of 12210
Processing 520 out of 12210
Processing 540 out of 12210
Processing 560 out of 12210
Processing 580 out of 12210
Processing 600 out of 12210
Processing 620 out of 12210
Processing 640 out of 12210
Processing 660 out of 12210
Processing 680 out of 12210
Processing 700 out of 12210
Processing 720 out of 12

In [264]:
BUILD_DATA = "files/build_data.csv"

df_build_data = pd.read_csv(BUILD_DATA)
print(f"Number of repo with github workflow action: {len(df_build_data)}")
df_build_data.head()

Number of repo with github workflow action: 11075


,repo,branch,default_branch,commit,workflow_name,run_id,status,conclusion,event,url,start_time,end_time,runner_environment
0,dvgis/dc-sdk,master,master,e810a4a289c04cbd66299fbf53d7e09f5ef7f873,build,17342172601,completed,success,push,https://github.com/dvt3d/dc-sdk/actions/runs/1...,2025-08-30T09:32:19Z,2025-08-30T09:34:17Z,NaN
1,chimoney/chimoney-community-projects,main,main,e27d3bc8a7211be50c265ed5b8353be85170a39f,PR Merge Badge Automation,16342288531,completed,skipped,pull_request,https://github.com/Chimoney/chimoney-community...,2025-07-17T10:09:03Z,2025-07-17T10:09:04Z,NaN
2,pony-house/client,dev,dev,bcb43f6e0a18f03e40dc058331a2b014ec367b0a,docker in /. - Update #1095030283,17571333539,completed,success,dynamic,https://github.com/Pony-House/Client/actions/r...,2025-09-09T04:01:05Z,2025-09-09T04:01:56Z,NaN
3,undershows/gigs,main,main,d7200937bec082870c02878b1e97a7c637cfe24b,npm_and_yarn in /. for devalue - Update #10972...,17636722869,completed,success,dynamic,https://github.com/undershows/gigs/actions/run...,2025-09-11T07:01:42Z,2025-09-11T07:03:18Z,NaN
4,volkswagen/github-app-authentication-action,main,main,7504f5f1674db96adf50d848508cb9d0ff32ef23,npm_and_yarn in /. - Update #983812978,13967443206,in_progress,NaN,dynamic,https://github.com/volkswagen/github-app-authe...,2025-03-20T10:44:43Z,2025-03-20T10:44:51Z,NaN


In [ ]:
workflow_details_from_ymls = []
repos_with_bot_workflows = []

def get_workflow_details_from_ymls():
    for _, row in df_build_data.iterrows():
        repo_name = row['repo'].lower()
        path = row['path']
        owner = repo_name.split('/')[0].strip()
        repo = repo_name.split('/')[1].strip()

        if(path.startswith('.github/workflows/') == False):
            repos_with_bot_workflows.append(f"{owner}/{repo}")
            continue
        
        yml_path = os.path.join("workflow_files", owner, repo, path.split('/')[-1])

        with open(yml_path, 'r') as f:

            try:
                yml = yaml.safe_load(f)

            except yaml.YAMLError as e:
                print(f"Error parsing {yml_path}: {e}")
                continue

            jobs = yml.get('jobs', {})

            jobs_info = []
            runs_on_set = set()

            for job_id, job in jobs.items():
                runs_on = job.get('runs-on', None)
                strategy = job.get('strategy', {})
                matrix = strategy.get('matrix', {})
                steps = job.get('steps', [])
                
                if isinstance(runs_on, list):
                    runs_on_set.update(runs_on)  # use update for lists
                else:
                    runs_on_set.add(runs_on)
                
                jobs_info.append({
                    "job_id": job_id,
                    "runs_on": runs_on,
                    "matrix": matrix,
                    "steps": steps
                }  )

            workflow_details_from_ymls.append({
                "repo": repo_name,
                "workflow_name": yml.get('name', '').strip(),
                "number_of_jobs": len(jobs),
                "highest_number_of_steps_in_a_job": max(len(job.get('steps', [])) for job in jobs.values()) if jobs else 0,
                "number_of_steps": sum(len(job.get('steps', [])) for job in jobs.values()),
                "jobs_info": jobs_info,
                "operating_systems": list(runs_on_set),
            })
    


In [249]:
#will fabricated
workflow_details_from_ymls = []
repos_with_missing_yml = []

def get_workflow_details_from_ymls():
    for _, row in df_build_data.iterrows():
        repo_name = row['repo'].lower()
        owner = repo_name.split('/')[0].strip()
        repo = repo_name.split('/')[1].strip()

        workflow_name = row['workflow_name']

        yml_path = os.path.join("workflow_files", owner, repo)
        yml_found = False

        for root, dirs, files in os.walk(yml_path):
            for file in files:
                file_path = os.path.join(root, file)

                if not file.endswith(('.yml', '.yaml')):
                    continue
                
                with open(file_path, 'r') as f:

                    try:
                        workflow = yaml.safe_load(f)

                    except yaml.YAMLError as e:
                        print(f"Error parsing {yml_path}: {e}")
                        continue

                    if workflow is None:
                        continue

                    if not isinstance(workflow, dict):
                        continue

                    if isinstance(workflow, dict):
                        if 'name' not in workflow or workflow.get('name') is None:
                            continue
            

                    if workflow.get('name', '') == workflow_name:
                        jobs = workflow.get('jobs', {})

                        jobs_info = []
                        os_set = set()

                        for job_id, job in jobs.items():
                            runs_on = job.get('runs-on', None)
                            strategy = job.get('strategy', {})
                            matrix = strategy.get('matrix', {})
                            steps = job.get('steps', [])
                            
                            if isinstance(runs_on, list):
                                os_set.update(runs_on)  # use update for lists
                            else:
                                os_set.add(runs_on)
                            os_set.discard(None)
                            
                            jobs_info.append({
                                "job_id": job_id,
                                "runs_on": runs_on,
                                "matrix": matrix,
                                "steps": steps
                            })

                        workflow_details_from_ymls.append({
                            "repo": repo_name,
                            "workflow_name": workflow_name,
                            "number_of_jobs": len(jobs),
                            "highest_number_of_steps_in_a_job": max(len(job.get('steps', [])) for job in jobs.values()) if jobs else 0,
                            "jobs_info": jobs_info,
                            "operating_systems": list(os_set),
                        })
                        yml_found = True
                        break

        if not yml_found:
            repos_with_missing_yml.append(f"{owner}/{repo}")
    


In [250]:
get_workflow_details_from_ymls()

Error parsing workflow_files/steve-xmh/amll-ttml-db: while constructing a mapping
  in "workflow_files/steve-xmh/amll-ttml-db/auto-untagged.yml", line 27, column 17
found unhashable key
  in "workflow_files/steve-xmh/amll-ttml-db/auto-untagged.yml", line 27, column 18
Error parsing workflow_files/reactplay/react-play: while parsing a block mapping
  in "workflow_files/reactplay/react-play/playwright-e2e.yml", line 35, column 11
expected <block end>, but found '-'
  in "workflow_files/reactplay/react-play/playwright-e2e.yml", line 36, column 11
Error parsing workflow_files/osc/ondemand: while scanning for the next token
found character '\t' that cannot start any token
  in "workflow_files/osc/ondemand/changelog.yml", line 24, column 38
Error parsing workflow_files/osc/ondemand: while scanning for the next token
found character '\t' that cannot start any token
  in "workflow_files/osc/ondemand/lint.yml", line 24, column 38
Error parsing workflow_files/osc/ondemand: while scanning for the

In [251]:
# workflow_details_from_ymls to dataframe
df_workflow_details_from_ymls = pd.DataFrame(workflow_details_from_ymls)
print(f"Number of workflow details from ymls: {len(df_workflow_details_from_ymls)}")


Number of workflow details from ymls: 8206


In [255]:
# header of df_workflow_details_from_ymls
df_workflow_details_from_ymls.head()

,repo,workflow_name,number_of_jobs,highest_number_of_steps_in_a_job,jobs_info,operating_systems
0,dvgis/dc-sdk,build,1,3,"[{'job_id': 'build', 'runs_on': 'ubuntu-latest...",[ubuntu-latest]
1,chimoney/chimoney-community-projects,PR Merge Badge Automation,1,4,"[{'job_id': 'badge_automation', 'runs_on': 'ub...",[ubuntu-latest]
2,mateonunez/website,ci,1,6,"[{'job_id': 'ci', 'runs_on': '${{ matrix.os }}...",[${{ matrix.os }}]
3,svg/svgo,CodeQL,1,4,"[{'job_id': 'analyze', 'runs_on': 'ubuntu-late...",[ubuntu-latest]
4,lupyuen/nuttx-ox64,Daily Test of NuttX for Ox64,1,6,"[{'job_id': 'test', 'runs_on': 'ubuntu-latest'...",[ubuntu-latest]


In [248]:
# df_workflow_details_from_ymls full details for repo 'quarto-dev/quarto-web'
df_workflow_details_from_ymls[df_workflow_details_from_ymls['repo'] == 'quarto-dev/quarto-web'].T

,8203
repo,quarto-dev/quarto-web
workflow_name,Update Downloads
number_of_jobs,3
highest_number_of_steps_in_a_job,4
jobs_info,"[{'job_id': 'update-downloads', 'runs_on': 'ub..."
operating_systems,"[None, ubuntu-latest]"


In [257]:
# merge df_ci_data and df_workflow_details_from_ymls on 'repo'
df_merged = pd.merge(df_build_data, df_workflow_details_from_ymls, on='repo', how='left', suffixes=('_ci', '_yml'))
df_merged

,repo,branch,default_branch,commit,workflow_name_ci,run_id,status,conclusion,event,url,start_time,end_time,runner_environment,workflow_name_yml,number_of_jobs,highest_number_of_steps_in_a_job,jobs_info,operating_systems
0,dvgis/dc-sdk,master,master,e810a4a289c04cbd66299fbf53d7e09f5ef7f873,build,17342172601,completed,success,push,https://github.com/dvt3d/dc-sdk/actions/runs/1...,2025-08-30T09:32:19Z,2025-08-30T09:34:17Z,NaN,build,1.0,3.0,"[{'job_id': 'build', 'runs_on': 'ubuntu-latest...",[ubuntu-latest]
1,chimoney/chimoney-community-projects,main,main,e27d3bc8a7211be50c265ed5b8353be85170a39f,PR Merge Badge Automation,16342288531,completed,skipped,pull_request,https://github.com/Chimoney/chimoney-community...,2025-07-17T10:09:03Z,2025-07-17T10:09:04Z,NaN,PR Merge Badge Automation,1.0,4.0,"[{'job_id': 'badge_automation', 'runs_on': 'ub...",[ubuntu-latest]
2,pony-house/client,dev,dev,bcb43f6e0a18f03e40dc058331a2b014ec367b0a,docker in /. - Update #1095030283,17571333539,completed,success,dynamic,https://github.com/Pony-House/Client/actions/r...,2025-09-09T04:01:05Z,2025-09-09T04:01:56Z,NaN,NaN,NaN,NaN,NaN,NaN
3,undershows/gigs,main,main,d7200937bec082870c02878b1e97a7c637cfe24b,npm_and_yarn in /. for devalue - Update #10972...,17636722869,completed,success,dynamic,https://github.com/undershows/gigs/actions/run...,2025-09-11T07:01:42Z,2025-09-11T07:03:18Z,NaN,NaN,NaN,NaN,NaN,NaN
4,volkswagen/github-app-authentication-action,main,main,7504f5f1674db96adf50d848508cb9d0ff32ef23,npm_and_yarn in /. - Update #983812978,13967443206,in_progress,NaN,dynamic,https://github.com/volkswagen/github-app-authe...,2025-03-20T10:44:43Z,2025-03-20T10:44:51Z,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11070,jsebrech/plainvanilla,main,main,745034afa11be22cf43ea97d2ddc54b1e1f5758f,Deploy static content to Pages,16693957140,completed,success,push,https://github.com/jsebrech/plainvanilla/actio...,2025-08-02T13:11:46Z,2025-08-02T13:12:04Z,NaN,Deploy static content to Pages,1.0,5.0,"[{'job_id': 'deploy', 'runs_on': 'ubuntu-lates...",[ubuntu-latest]
11071,quarto-dev/quarto-cli,main,main,1ca3dabc13786e3bf91b70eb3333185c799b3414,Performance Check,17654586346,completed,success,schedule,https://github.com/quarto-dev/quarto-cli/actio...,2025-09-11T19:03:00Z,2025-09-11T19:04:29Z,NaN,Performance Check,1.0,7.0,"[{'job_id': 'test-bundle', 'runs_on': 'ubuntu-...",[ubuntu-latest]
11072,quarto-dev/quarto-web,main,main,88fff8bb9f3827f0b1c577335424c8482eb554c3,Update Downloads,17655026497,completed,success,schedule,https://github.com/quarto-dev/quarto-web/actio...,2025-09-11T19:21:10Z,2025-09-11T19:21:43Z,NaN,Update Downloads,3.0,4.0,"[{'job_id': 'update-downloads', 'runs_on': 'ub...",[ubuntu-latest]
11073,newan/iobroker.bluelink,master,master,40ac86540cbf5ac723eb7b96cc56d3cd0e7320aa,CodeQL,17530721044,completed,success,schedule,https://github.com/Newan/ioBroker.bluelink/act...,2025-09-07T15:56:06Z,2025-09-07T15:57:21Z,NaN,CodeQL,1.0,4.0,"[{'job_id': 'analyze', 'runs_on': 'ubuntu-late...",[ubuntu-latest]


In [266]:
df_merged.to_csv("files/merged_ci_workflow_details.csv", index=False)

In [ ]:
# iterate items in operating_systems column and remove entries that contain special characters like $ { { } }
df_merged['operating_systems'] = df_merged['operating_systems'].apply(lambda os_list: [os for os in os_list if all(c.isalnum() or c in ('-', '_', '.') for c in os)] if isinstance(os_list, list) else os_list)